# COVID-19 Dashboard

This dashboard shows the latest data on COVID-19(Coronavirus), regarding how it is affecting the UK.

In [2]:
from uk_covid19 import Cov19API
import matplotlib.pyplot as plt
import pandas as pd
import json
import matplotlib.animation as animation
from matplotlib import style
import ipywidgets as wdg
from datetime import datetime
from IPython.display import display
output=wdg.Output()

style.use('dark_background')
plt.rcParams['figure.dpi'] = 100

The purpose of this graph is to show cases that can be described as serious. The two types of cases seen on the graph are death, and cases requiring hospitalisation. You can select either of these options on from the list below to view the data seperately. 

In [3]:
#load saved data
with open("seriouscases.json", "rt") as INFILE:
    data=json.load(INFILE)

#define key functions
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def convert_date(datestring):
    #Convert a date string into a pandas datetime object
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangledata(x):
    global seriouscasesdf
    #wrangle dates
    datalist=x['data']
    dates=[dictionary['date'] for dictionary in datalist]
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    #print(startdate, ' to ', enddate)
    
    #make dataframe
    index=pd.date_range(startdate, enddate, freq='D')
    seriouscasesdf=pd.DataFrame(index=index, columns=['hospitalCases', 'deaths'])

    #fill dataframe
    for entry in datalist: 
        date=convert_date(entry['date'])
        for column in ['hospitalCases', 'deaths']:
            if pd.isna(seriouscasesdf.loc[date, column]): 
                value = float(entry[column]) if entry[column]!=None else 0.0
                seriouscasesdf.loc[date, column]=value
    seriouscasesdf.fillna(0.0, inplace=True)
    return(seriouscasesdf)

#update implementation
def access_api():
    #this will overwrite the previous json file
    try:
        global data
        filters = [
            'areaType=overview'
        ]

        structure = {
            "date": "date",
            "hospitalCases":"hospitalCases",
            "deaths":"newDeaths28DaysByPublishDate"
        }

        api = Cov19API(filters=filters, structure=structure)
        seriouscases=api.get_json()

        with open("seriouscases.json", "wt") as OUTF:
            json.dump(seriouscases, OUTF)
        with open("seriouscases.json", "rt") as INFILE:
            data=json.load(INFILE)
        return(data)

    except:
        with output:
            print('Error getting new data from PHE')


def api_button_callback(button):
    apidata=access_api()
    apidata2=access_api2()
    global seriouscasesdf
    global cumulativesdf
    seriouscasesdf=wrangledata(apidata)
    cumulativesdf=wrangledata2(apidata2)
    refresh_graph()
    refresh_graph2()
    apibutton.icon="check"
    apibutton.disabled=True


apibutton=wdg.Button(
    description='UPDATE',
    disabled=False,
    button_style='info',
    tooltip="Refreshes the graph with the latest data from Public Health England",
    icon=''
)


apibutton.on_click(api_button_callback)
display(apibutton)
display(output)

wrangledata(data)

#plot graph no.1

def refresh_graph():
    Case.disabled=True
    Case.disabled=False
    

def plot_serio(x):
    ncols=len(x)
    if ncols>0:
        a=seriouscasesdf[list(x)].plot()
        a.grid(color='grey',linewidth=0.5)
    else:
        # these messages will be printed in no column is selected
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

Case=wdg.SelectMultiple(
    options=['hospitalCases','deaths'],
    value=['hospitalCases','deaths'],
    rows=2,
    description='Show: ',
    disabled=False,
)
    
graph=wdg.interactive_output(plot_serio, {'x': Case})
display(Case, graph)



Button(button_style='info', description='UPDATE', style=ButtonStyle(), tooltip='Refreshes the graph with the l…

Output()

SelectMultiple(description='Show: ', index=(0, 1), options=('hospitalCases', 'deaths'), rows=2, value=('hospit…

Output()

Here you can select between viewing cumulative totals of: cases, hospital admissions, and deaths as they have increased since the start of the pandemic. 

In [160]:
        
def access_api2():
    #this will overwrite the previous json file
    try: 
        filters = [
            'areaType=overview'
        ]

        structure = {
            "date": "date",
            "CumulativeCases":"cumCasesByPublishDate",
            "CumulativeAdmissions":"cumAdmissions",
            "CumulativeDeaths":"cumDeaths28DaysByDeathDate"
        }

        api = Cov19API(filters=filters, structure=structure)
        cumulatives=api.get_json()

        with open("cumulatives.json", "wt") as OUTF:
            json.dump(cumulatives, OUTF)
        with open("cumulatives.json", "rt") as INFILE:
            data2=json.load(INFILE)

        return(data2)
    except:
        with output:
            print('Error getting new data from PHE')


def wrangledata2(x):
    global cumulativesdf
    #wrangle dates
    datalist=x['data']
    dates=[dictionary['date'] for dictionary in datalist]
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    #print(startdate, ' to ', enddate)
    
    #make dataframe
    index=pd.date_range(startdate, enddate, freq='D')
    cumulativesdf=pd.DataFrame(index=index, columns=['CumulativeCases','CumulativeAdmissions','CumulativeDeaths'])

    #fill dataframe
    for entry in datalist: 
        date=convert_date(entry['date'])
        for column in ['CumulativeCases','CumulativeAdmissions','CumulativeDeaths']:
            if pd.isna(cumulativesdf.loc[date, column]): 
                value = float(entry[column]) if entry[column]!=None else 0.0
                cumulativesdf.loc[date, column]=value
    cumulativesdf.fillna(0.0, inplace=True)
    return(cumulativesdf)

colour={'CumulativeCases':'lightseagreen','CumulativeAdmissions':'khaki','CumulativeDeaths':'lightcoral'
       }
thickness={'CumulativeCases':4,'CumulativeAdmissions':4,'CumulativeDeaths':4
       }
Title={'CumulativeCases':'Cumulative Cases','CumulativeAdmissions':'Cumulative Admissions','CumulativeDeaths':'Cumulative Deaths'
       }

with open("cumulatives.json", "rt") as INFILE:
    data2=json.load(INFILE)
    
wrangledata2(data2)

def plot_cumu(x):
    ax2=cumulativesdf[x].plot(color=(colour[x]),linewidth=(thickness[x]))
    ax2.set_title(Title[x], fontsize=20)
    ax2.grid(color='grey',linewidth=0.5)
    
Selection=wdg.Dropdown(
    options=['CumulativeCases','CumulativeAdmissions', 'CumulativeDeaths'],
    value='CumulativeDeaths',
    description='Show: ',
    disabled=False,
)

def refresh_graph2():
    current=Selection.value
    if current==Selection.options[0]:
        other=Selection.options[1]
    else:
        other=Selection.options[0]
    Selection.value=other # forces the redraw
    Selection.value=current

graph=wdg.interactive_output(plot_cumu, {'x': Selection})
display(Selection, graph)


Dropdown(description='Show: ', index=2, options=('CumulativeCases', 'CumulativeAdmissions', 'CumulativeDeaths'…

Output()

This Dashboard was made by Edward Monah based on code from Fabrizio Smeraldi

UK Government data published by Public Health England has been used.